# Curve Shortening

This lab is designed to give you a little practice with the curve shortening algorithms from section 5.5 in the textbook. 

## Library code: 

Start by executing the cell below to load the library code: 

In [1]:
from koebe.geometries.euclidean2 import PointE2, SegmentE2

########################################
# Data structures: Vertex, Polygon
########################################

class Vertex:
    """
        Simple Vertex class for representing a polygon as a linked list of 
        vertices. Each vertex stores .next and .prev pointers as well as a .pos
        PointE2 object that is the vertex's position in the plane.
    """
    def __init__(self, pos: PointE2):
        """
        Initializes the vertex at the given position.
        
        Attributes:
            pos: PointE2 - The position to initialize this vertex to.
        """
        self.pos = pos
        self.prev = None
        self.next = None
    
    def setNextVertex(self, nextVertex: "Vertex"):
        """
        Makes nextVertex the next vertex from this one. 
        """
        self.next = nextVertex
        nextVertex.prev = self

class Polygon:
    """
        Simple polygon class that is a container for a linked list of vertices. 
        The class is iterable over the vertices. 
        
        Here are a few examples of use for a Polygon object named poly: 
        
        allVertices = [v for v in poly]
        allVertexPos = [v.pos for v in poly] # get the 2D euclidean points for the verts in order
        allVertexPos = poly.vertexPositions() # convenience method doing the same as above
        
        xs = [v.pos.x for v in poly] # Get just the x coordinates of the vertices.
        ys = [v.pos.y for v in poly] # Get just the y coordinates of the vertices.
        
    """
    def __init__(self, vertexPositions = []):
        if len(vertexPositions) > 0:
            vertices = [Vertex(p) for p in vertexPositions]
            for i in range(len(vertices)):
                vertices[i - 1].setNextVertex(vertices[i])
            self.firstVertex = vertices[0]
        else:
            self.firstVertex = None
    
    def __iter__(self):
        """
        An iterator for the polygon that allows you to loop
        over a polygon's vertices using: 
        
        for vertex in polygon:
            ... do something with the vertex object ...
        """
        current = self.firstVertex
        while True:
            yield current
            current = current.next
            if current is self.firstVertex:
                break
    
    def vertexPositions(self):
        """
        Returns the positions of all ther vertices in order.
        """
        return [v.pos for v in self] # Uses the __iter__ to build a list
    
    def copy(self):
        """
        Returns a copy of this polygon. This performs a shallow copy
        since PointE2 objects are immutable. 
        """
        return Polygon(self.vertexPositions())

########################################
# Graphics
########################################
    
from koebe.geometries.euclidean2 import PointE2
from ipycanvas import Canvas, hold_canvas

def CurveShorteningCanvas(scaling = False):

    canvas = Canvas(size=(600,600))
    
    points = [PointE2(100, 200), PointE2(200, 200), 
              PointE2(200, 300), PointE2(200, 400), 
              PointE2(300, 200), PointE2(300, 300), 
              PointE2(300, 400), PointE2(400, 400), 
              PointE2(500, 100)]
    
    per = perimeter(Polygon(points))
    
    selectedPointIdx = -1
    buttonDown = False
    
    currentMode = "Edit"
    
    def nextModeGenerator():
        nonlocal currentMode
        while True:
            currentMode = "Median"; yield
            currentMode = "Discrete flow"; yield
            currentMode = "Edit"; yield
    
    mode = nextModeGenerator()

    def draw():
        nonlocal canvas, points, selectedPointIdx, buttonDown, currentMode
        with hold_canvas(canvas):
            canvas.clear()
            
            canvas.begin_path()
            canvas.stroke_style = 'black'
            canvas.move_to(int(points[-1].x), int(points[-1].y))
            for p in points:
                canvas.line_to(int(p.x), int(p.y))
            canvas.stroke()
            
            canvas.fill_style = 'blue'
            canvas.fill_rects([int(p.x) - 4 for p in points], 
                               [int(p.y) - 4 for p in points], 
                               8)
            if selectedPointIdx != -1:
                canvas.fill_style = 'red'
                canvas.fill_rect(int(points[selectedPointIdx].x) - 4, 
                                  int(points[selectedPointIdx].y) - 4, 
                                  8)
            
            canvas.fill_style = "#aaf" if not buttonDown else "#77e"
            canvas.fill_rect(10, 10, 220, 40)
            canvas.fill_style = "#000"
            canvas.font = '24px serif'
            canvas.fill_text(currentMode, 20, 38)

    def handle_mouse_down(x, y):
        nonlocal points, selectedPointIdx, buttonDown, currentMode
        # See if any point is close to x, y
        if x >= 10 and x <= 230 and y >= 10 and y <= 50:
            buttonDown = True
        elif currentMode == "Edit":
            cursorPoint = PointE2(x, y)
            sqDists = [p.distSqTo(cursorPoint) for p in points]
            minIdx = sqDists.index(min(sqDists))
            if sqDists[minIdx] < 24:
                selectedPointIdx = minIdx
        draw()

    def handle_mouse_up(x, y):
        nonlocal selectedPointIdx, buttonDown, mode, points, per, scaling
        if buttonDown and x >= 10 and x <= 230 and y >= 10 and y <= 50:
            next(mode)
        elif currentMode == "Discrete flow":
            if scaling:
                points = scale_to_length(discreteflow_smooth(Polygon(points)), per).vertexPositions()
            else: 
                points = discreteflow_smooth(Polygon(points)).vertexPositions()
        elif currentMode == "Median":
            if scaling:
                points = scale_to_length(median_smooth(Polygon(points)), per).vertexPositions()
            else:
                points = median_smooth(Polygon(points)).vertexPositions()
        buttonDown = False
        selectedPointIdx = -1 # No point is selected anymore.
        draw()

    def handle_mouse_move(x, y):
        nonlocal points, selectedPointIdx
        if selectedPointIdx >= 0 and (x > 230 or y > 50 or x < 10 or y < 10):
            points[selectedPointIdx] = PointE2(x, y)
        draw()

    canvas.on_mouse_down(handle_mouse_down)
    canvas.on_mouse_up(handle_mouse_up)
    canvas.on_mouse_move(handle_mouse_move)
    
    draw()
    
    return canvas

# Your Task

1. Execute the code below. Notice that in addition to an editable polygon, you also get a blue button that currently says "Edit". If you click this button it will cycle through "Edit", "Median", and "Discrete flow" modes. Edit mode allows you to edit the vertices of the polygon. The median and discretre flow modes will execute your smoothing functions once you have implemented them. In either median or discrete flow mode clicking anywhere on the polygon will execute one smoothing step. I have implemented a simple scaling function to show you how this works (`median_smooth` will shrink and `discreteflow_smooth` will grow the polygon). 
2. Replace my dummy implementation of `median_smooth` with an implementation of the algorithm as described in section 5.5. Recall that the median of a line segment from $(x_0, y_0)$ to $(x_1, y_1)$ is $(\frac{x_0 + x_1}{2}, \frac{y_0 + y_1}{2})$. 
3. Once you have your code working for `median_smooth`, play with a few examples and answer the questions below. 

  __Q:__ Can you find an example where the median_smooth operation ends up creating a self-intersecting polygon. 
  
  _Your answer here:_
  
  __Q:__ If you run the median smooth enough times in a row, what happens to the polygon?
  
  _Your answer here:_
  
4. Replace my dummy implementation of `discreteflow_smooth`with an implementation of the discrete flow algorithm from section 5.5. 
5. Once you have your code working for `discreteflow_smooth`, play with a few examples and answer the questions below. 

  __Q:__ Can you find an example where the median_smooth operation ends up creating a self-intersecting polygon. 
  
  _Your answer here:_
  
  __Q:__ If you run the median smooth enough times in a row, what happens to the polygon?
  
  _Your answer here:_
  
6. You should have notice that in both flows the polygons eventually shrink to zero perimeter. We can fix this by scaling the polygon after each step so that its perimeter does not change. To do this, you need to implement the `perimeter`, `centroid`, and `scale_to_length` functions below. Change the call to `CurveShorteningCanvas` so that `scaling = True` to see the result of your work. 

# Grading: 

* For a 6/10 on this assignment, complete steps 1-3. 
* For an 8/10 on this assignment, complete steps 1-5. 
* For a 10/10 on this assignment, complete all steps. 

In [8]:
def median_smooth(poly:Polygon) -> Polygon:
    """
    Input: A Polygon object. 
    Output: A new Polygon object where the vertex at index i in the output polygon is the 
            midpoint of the line segment from vertex (i-1) to verex i in the input polygon.
    """
    # TODO Replace this:
    return Polygon([
        PointE2(v.pos.x / 1.01, v.pos.y / 1.01)
        for v in poly
    ])

def discreteflow_smooth(poly: Polygon, delta: float = 0.1) -> Polygon:
    """
    Input: A Polygon object poly and a real number delta.  
    Output: A new Polygon object where the vertex at index i has been moved according to
            the discrete curve shortening algorithm described in section 5.5 of your book.
    Hint: Remember that each vertex in the polygon stores .next and .prev pointers.
    """
    # TODO Replace this:
    return Polygon([
        PointE2(v.pos.x * 1.01, v.pos.y * 1.01)
        for v in poly
    ])

def perimeter(poly: Polygon) -> float:
    """
    Input: A Polygon object. 
    Output: The perimeter of the polygon. 
    Hint: Remember that PointE2 has distTo and each vertex v in poly has .next and .prev.
    """
    # TODO Replace this:
    return 42

def centroid(poly: Polygon) -> PointE2:
    """
    Input: A Polygon object. 
    Output: A PointE2 object that is the center of mass of the vertices. 
    Hint: If xSum is the sum of the x-coordinates, 
          and ySum is the sum of the y-coordinates, and the polygon has N vertices, 
          then the centroid is at (xSum / N, ySum / N). 
    """
    # TODO Replace this:
    return PointE2(0, 0)

def scale_to_length(poly: Polygon, length: float) -> Polygon:
    """
    Take each vertex v of the polygon and move it outwards along the ray from the centroid 
    towards v so that the perimeter of the returned polygon is equal to the length parameter. 
    
    Input: 
        poly: Polygon - The polygon to scale
        length: float - The desired perimeter length. 
    Output:
        A new Polygon object whose perimeter is equal to length. 
    
    Hint: 
        Scaling factor should be length / perimeter. Compute the vector V from the centroid 
        to each vertex v's position. Then scale V by length / perimeter to obtain a new vector
        W. Then add W to the centroid. 
    """
    # TODO Replace this:
    return Polygon([PointE2(v.pos.y, v.pos.x) for v in poly])


CurveShorteningCanvas(scaling = True)

Canvas(height=600, width=600)